<a href="https://colab.research.google.com/github/JDM-1609/Statistical-Process-Control-in-Injection-Machines/blob/main/SPC-BASED%20CONTROL%20PLAN%20OPTIMIZER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**OPTIMIZADOR DE PLANES DE CONTROL BASADO EN SPC**

**CONFIGURACIONES INICIALES**

In [ ]:
# Carga de archivos CSV para análisis
from google.colab import files

uploaded = files.upload()
RUTA_IN = next(iter(uploaded.keys()))
print("Archivo cargado:", RUTA_IN)

In [ ]:
# Importación de recursos necesarios para el desarrollo
import csv
from pathlib import Path
from io import StringIO
from ipywidgets import IntSlider, interact, Dropdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Configuraciones para mejorar la visualización de los datos en los DF
pd.set_option("display.max_columns", 20) # Controla cuántas columnas se muestran sin omisiones
pd.set_option("display.max_row", None)
pd.set_option("display.float_format", lambda v: f"{v:.5g}") # Muestra valores con hasta 5 cifras significativas
ENCODING_ALS = "CP1250" # Encoding de los archivos provenientes ALS

#### **VISUALIZACIÓN DE DATOS Y GRÁFICAS DE CONTROL MEDIANTE SPC**

Este módulo recibe los archivos generados apartir del sistema ALS, separando el DataFrame en diferentes secciones especificadas (Info. Contextual, estadísticos y datos crudos) creando un DataFrame para cada una, permitiendo extraer datos de interés para llevar trazabilidad de los cambios que se evidencian o realizan durante la optimización de los límites de control de los planes de control de las inyectoras en la empresa SIMEX. Con lo anterior, se logra contruir las gráficas de control (I, X̄ y S) para la identificación de mediciones fuera de especificaciones.

In [ ]:
# Inicialización de índices (start & end) de cada sección
# SECCIONES
idx_info = 0           # Información contextual
idx_stats_start = 2    # Start Estadísticos
idx_stats_end = 21     # End Estadísticos
idx_raw_start = 23     # Datos crudos

FUNCIONES DEL VISUALIZADOR

In [ ]:
# @title
# FUNCIONES: División de las secciones en un DF diferente cada uno.

# Lee el CSV como strings y devuelve una lista donde cada elemento es una línea del CSV completo
def leer_lineas(ruta_csv: str, encoding: str = ENCODING_ALS):
    raw = Path(ruta_csv).read_text(encoding=encoding, errors="replace")
    lines = raw.splitlines()  # Este método siempre devuelve una lista
    return lines

# Devuelve la sección "Información Contextual" como un DF
def parse_info_contextual(lines, idx_info: int = 0) -> pd.DataFrame:
    line = lines[idx_info]  # Se toma únicamente la primera línea
    cells = next(csv.reader([line], delimiter=",", quotechar='"'))

    # Limpieza eliminando vacíos y el elemento "Valores reales"
    cleaned = []
    for c in cells:
        c = c.strip().strip('"')
        if not c:  # Si está vacío -> se omite
            continue
        if c.lower() == "valores reales":
            continue
        cleaned.append(c)

    claves  = cleaned[0::2]
    valores = cleaned[1::2]

    df_info = pd.DataFrame([valores], columns=claves)
    return df_info

# Devuelve la sección "Estadísticos" como un DF
def parse_estadisticos(lines, idx_start: int, idx_end: int) -> pd.DataFrame:

    # Se unen las líneas de la sección en un solo string tipo CSV
    block = "\n".join(lines[idx_start:idx_end])

    # Leemos este bloque como si fuera un CSV independiente
    df_stats = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
        )
    # Los índices del df son los nombres de los estadísticos
    df_stats.index.name = "Estadísticos"

    # Ajuste de los encabezados de las columnas
    cols = list(df_stats.columns)
    if str(cols[0]).strip() == "":
      df_stats.columns = cols[1:] + cols[:1] # Se desplazan las columnas a la izquierda
      df_stats = df_stats.dropna(axis=1, how="all") # Elimina la columna vacía

    return df_stats

# Devuelve la sección "Datos Crudos" como un DF
def parse_datos_crudos(lines, idx_start: int) -> pd.DataFrame:

    # Se unen las líneas desde idx_start hasta final de los datos
    block = "\n".join(lines[idx_start:])

    df_raw = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
    )
    df_raw.columns = [str(c).strip() for c in df_raw.columns]
    df_raw = df_raw.reset_index()

    cols = list(df_raw.columns)
    df_raw.columns = cols[1:] + cols[:1] # Se desplazan las columnas a la izq

    # Eliminar columnas completamente vacías (NaN en todas las filas)
    df_raw = df_raw.dropna(axis=1, how="all")

    return df_raw

"------------------------------------------------------------------------------"

# FUNCIONES: Extracción de los datos estadísticos de interés para los seguimientos

# Redondeo a 5 cifras significativas
def sig(x, n=5):
    try:
        return float(f"{x:.{n}g}")
    except:
        return x

# Construcción de la tabla final para seguimientos
def estadisticos_resumen(df_stats):

    variables = df_stats.columns.tolist()
    filas = []

    for var in variables:
        try:
            # Extrae valores de df_stats
            valor_nominal = df_stats.loc["Valor nominal", var]
            xqq           = df_stats.loc["xqq", var]
            sigma         = df_stats.loc["Sigma", var]
            cp            = df_stats.loc["Cp", var]
            cpd           = df_stats.loc["Cpd", var]

            # Calcula Rango
            rango = df_stats.loc["Tolerancia superior", var] - valor_nominal

            # Calcula Desviación
            desviacion = ((cp - cpd) / (cp))*100 if cp != 0 else np.nan

            # Convierte a 3 cifras significativas cada valor de interés
            fila = [
                var,
                sig(rango),
                sig(valor_nominal),
                sig(xqq),
                sig(sigma),
                sig(cp),
                sig(cpd),
                sig(desviacion)
            ]
        # Si se encuentran valores faltantes devuelve toda la fila como "NaN"
        except KeyError:
            fila = [var] + [np.nan]*7

        filas.append(fila)

    # Se crea el DF final
    df_resumen = pd.DataFrame(
        filas,
        columns=[
            "Variables", "Rango", "Valor nominal", "Media",
            "Sigma_W", "Cp", "Cpk", "Desviación"
        ]
    )

    return df_resumen

"------------------------------------------------------------------------------"

# FUNCIONES: Clasificación de datos por subgrupos y cálculos SPC

# De datos crudos se clasifica para n=5 mediciones por subgrupo y se realizan cálculos SPC
def procesar_datos_spc(df_raw):

    df = df_raw.copy()

    # Se separa el subgrupo con la medición en dos columnas distintas
    df[["subgrupo", "pos"]] = df["Muestra aleatoria"].str.split("/", expand=True)
    df["subgrupo"] = df["subgrupo"].astype(int)
    df["pos"] = df["pos"].astype(int)

    # Toma solo las columnas con valores númericos (Mediciones de cada variable de control)
    columnas_no_variables = ["Muestra aleatoria", "Momento", "Cantidad piezas", "subgrupo", "pos"]
    variables = [c for c in df.columns if c not in columnas_no_variables]

    # Calcular X̄ y S por subgrupo
    df_xbar = df.groupby("subgrupo")[variables].mean()   # Media por subgrupo
    df_s = df.groupby("subgrupo")[variables].std(ddof=1) # Desv std por subgrupo

    xbar_prom_global = df_xbar.mean() # Cálculo de la media de todas las medias
    s_prom_global = df_s.mean()       # Cálculo de la media de las desv std

    return df, df_xbar, df_s, xbar_prom_global, s_prom_global

"------------------------------------------------------------------------------"
# GRÁFICAS
# FUNCIONES: Generación de la carta I por variable controlada

# Grafica con slider con identificación de puntos fuera de tolerancia
# Menú desplegable para seleccionar la variable de interés

def graficar_carta_i(df_ind, df_stats, variable, window=20, titulo=None):
    # Datos base
    vals_ind = df_ind[variable].astype(float).to_numpy()     # valores individuales
    n_obs = len(vals_ind)
    muestras = np.arange(1, n_obs + 1)                       # índice de muestra 1,...,N

    # Valor nominal desde df_stats
    valor_nominal = float(df_stats.loc["Valor nominal", variable])

    # Límites de especificación
    USL = float(df_stats.loc["Tolerancia superior", variable])
    LSL = float(df_stats.loc["Tolerancia inferior", variable])

    # Límites de control de individuales
    UCLi = float(df_stats.loc["LISx", variable])
    LCLi = float(df_stats.loc["LIIx", variable])

    # Escala eje Y automática
    all_vals = np.concatenate([vals_ind, [USL, LSL, UCLi, LCLi, valor_nominal]])
    y_min = all_vals.min()
    y_max = all_vals.max()
    margen = 0.05 * (y_max - y_min if y_max > y_min else 1.0)
    y_min -= margen
    y_max += margen

    # Ventana deslizante
    if window > n_obs:
        window = n_obs

    if titulo is None:
        titulo = (f"Carta de Individuales - {variable}   "
                  f"VN={valor_nominal:.3f}   USL={USL:.3f}   LSL={LSL:.3f}")

    def plot_window(start_idx: int):
        end_idx = min(start_idx + window, n_obs)
        x_win = muestras[start_idx:end_idx]
        y_win = vals_ind[start_idx:end_idx]

        plt.figure(figsize=(12, 5))
        ax = plt.gca()

        # Zonas fuera de especificación
        ax.axhspan(USL, y_max, facecolor="red", alpha=0.08)
        ax.axhspan(y_min, LSL, facecolor="red", alpha=0.08)

        # Serie de individuales
        ax.plot(
            x_win, y_win,
            marker="o",
            linestyle="-",
            color="blue",
            label="Medición"
        )

        # Puntos fuera de especificación
        mask_out = (y_win > USL) | (y_win < LSL)
        ax.scatter(
            x_win[mask_out],
            y_win[mask_out],
            s=50,
            color="red",
            edgecolors="black",
            zorder=5,
            label="Fuera de especificación" if mask_out.any() else None
        )

          # Valor nominal (VERDE)
        ax.axhline(
            y=valor_nominal,
            linestyle="-",
            linewidth=1.2,
            color="green",
            label="Valor nominal"
        )
         # Límites de especificación (rojo)
        ax.axhline(
            y=USL,
            linestyle="-",
            linewidth=1.2,
            color="red",
            label="USL"
        )
        ax.axhline(
            y=LSL,
            linestyle="-",
            linewidth=1.2,
            color="red",
            label="LSL"
        )

        # Límites de control de individuales (naranja punteado)
        ax.axhline(
            y=UCLi,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="UCL"
        )
        ax.axhline(
            y=LCLi,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="LCL"
        )

        # Ejes y título
        ax.set_xlabel("Número de muestra")
        ax.set_ylabel(variable)
        ax.set_title(titulo)

        # Eje X acotado a la ventana actual
        ax.set_xlim(x_win.min() - 0.5, x_win.max() + 0.5)
        ax.set_xticks(x_win)

        # Escala vertical
        ax.set_ylim(y_min, y_max)

        # Grid
        ax.grid(True, linestyle="--", alpha=0.4)

        # Marco exterior negro
        for spine in ax.spines.values():
            spine.set_edgecolor("black")
            spine.set_linewidth(1.2)

        # Leyenda fuera de la gráfica
        ax.legend(
            loc="upper center",
            bbox_to_anchor=(0.5, -0.20),
            ncol=6,
            frameon=False,
            fontsize=9
        )

        plt.tight_layout()
        plt.show()

    # Slider sobre las muestras individuales
    max_start = max(n_obs - window, 0)
    slider = IntSlider(
        value=0,
        min=0,
        max=max_start,
        step=1,
        description="Inicio:",
        continuous_update=False,
        )

    interact(plot_window, start_idx=slider)
def selector_carta_i(df_ind, df_stats, window=200):

    variables = [c for c in df_ind.columns
                 if c not in ["Muestra aleatoria", "Momento", "Cantidad piezas", "subgrupo", "pos"]]

    def _plot(variable):
        graficar_carta_i(df_ind, df_stats, variable, window=window)

    interact(_plot, variable=Dropdown(options=variables, description="Variable:"))

"------------------------------------------------------------------------------"

# FUNCIONES: Generación de la carta X̄ por variable controlada

def graficar_carta_x(df_xbar, xbar_prom, df_stats, variable, window=30, titulo=None):

    # Datos base
    xbar_subgrupos = df_xbar[variable]                      # Serie con X̄ por subgrupo
    subgrupos = xbar_subgrupos.index.astype(int).to_numpy() # Índices de subgrupos como enteros

    xbar_global = float(xbar_prom[variable])

    # Límites de especificación
    USLx = float(df_stats.loc["Tolerancia superior", variable])
    LSLx = float(df_stats.loc["Tolerancia inferior", variable])

    # Límites de control
    UCLx = float(df_stats.loc["LISx", variable])
    LCLx = float(df_stats.loc["LIIx", variable])

    # Escala eje Y automática
    all_vals = np.concatenate([
        xbar_subgrupos.values,
        [USLx, LSLx, UCLx, LCLx, xbar_global]
    ])
    y_min = all_vals.min()
    y_max = all_vals.max()
    margen = 0.05 * (y_max - y_min if y_max > y_min else 1.0)
    y_min -= margen
    y_max += margen

    # Ventana deslizante
    n = len(subgrupos)
    if window > n:
        window = n

    if titulo is None:
        titulo = (f"Carta X̄ - {variable}   "
                  f"SW={xbar_global:.3f}   "
                  f"USL={USLx:.3f}   LSL={LSLx:.3f}")

# FUNCIÓN: Configuración gráfica
    def plot_window(start_idx: int):
        end_idx = min(start_idx + window, n)
        sg = subgrupos[start_idx:end_idx]
        vals = xbar_subgrupos.iloc[start_idx:end_idx]

        plt.figure(figsize=(12, 5))
        ax = plt.gca()

        # Zonas fuera de especificación (rojo)
        ax.axhspan(USLx, y_max, facecolor="red", alpha=0.08)
        ax.axhspan(y_min, LSLx, facecolor="red", alpha=0.08)

        # Serie X̄
        ax.plot(
            sg, vals,
            marker="o",
            linestyle="-",
            color="blue",
            label="X̄ por subgrupo"
        )

        # Puntos fuera de especificación
        mask_out = (vals > USLx) | (vals < LSLx)
        ax.scatter(
            sg[mask_out],
            vals[mask_out],
            s=50,
            color="red",
            edgecolors="black",
            zorder=5,
            label="Fuera de especificación" if mask_out.any() else None
        )

        # Media global (verde punteada)
        ax.axhline(
            y=xbar_global,
            linestyle="--",
            linewidth=0.8,
            color="green",
            label="X̄ global"
        )

        # Límites de especificación (rojo)
        ax.axhline(
            y=USLx,
            linestyle="-",
            linewidth=1.2,
            color="red",
            label="USLx"
        )
        ax.axhline(
            y=LSLx,
            linestyle="-",
            linewidth=1.2,
            color="red",
            label="LSLx"
        )

        # Límites de control (naranja punteado)
        ax.axhline(
            y=UCLx,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="UCLx"
        )
        ax.axhline(
            y=LCLx,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="LCLx"
        )

        # Ejes y título
        ax.set_xlabel("Subgrupos")
        ax.set_ylabel(variable)
        ax.set_title(titulo)

        # Eje X acotado a la ventana actual
        ax.set_xlim(sg.min() - 0.5, sg.max() + 0.5)
        ax.set_xticks(sg)

        # Escala vertical
        ax.set_ylim(y_min, y_max)

        # Grid
        ax.grid(True, linestyle="--", alpha=0.4)

        # Marco exterior negro
        for spine in ax.spines.values():
            spine.set_edgecolor("black")
            spine.set_linewidth(1.2)

        # Leyenda fuera de la gráfica
        ax.legend(
            loc="upper center",
            bbox_to_anchor=(0.5, -0.20),
            ncol=6,
            frameon=False,
            fontsize=9
        )

        plt.tight_layout()
        plt.show()

    # Slider para elegir la ventana de subgrupos
    max_start = max(n - window, 0)
    slider = IntSlider(
        value=0,
        min=0,
        max=max_start,
        step=1,
        description="Inicio:",
        continuous_update=False,
    )

    interact(plot_window, start_idx=slider)

# FUNCIÓN: Menú desplegable para selección de la variable de interés
def selector_carta_x(df_xbar, xbar_prom, df_stats, window=30):

    # Lista de variables disponibles
    variables = list(df_xbar.columns)

    def _plot(variable):
        graficar_carta_x(df_xbar, xbar_prom, df_stats, variable, window=window)

    interact(_plot, variable=Dropdown(options=variables, description="Variable:"))

"------------------------------------------------------------------------------"

# FUNCIONES: Generación de la carta S por variable controlada
def graficar_carta_s(df_s, s_prom, df_stats, variable, window=30, titulo=None):
    # Datos base
    s_subgrupos = df_s[variable]                          # Serie S por subgrupo
    subgrupos = s_subgrupos.index.astype(int).to_numpy()  # Índices de subgrupos como enteros

    s_global = float(s_prom[variable])

    # Límites de control de S
    UCLs = float(df_stats.loc["LISs", variable])   # Límite superior de control de S
    LCLs = float(df_stats.loc["LIIs", variable])   # Límite inferior de control de S

    # Escala eje Y automática
    all_vals = np.concatenate([
        s_subgrupos.values,
        [UCLs, LCLs, s_global]
    ])
    y_min = all_vals.min()
    y_max = all_vals.max()
    margen = 0.05 * (y_max - y_min if y_max > y_min else 1.0)
    y_min -= margen
    y_max += margen

    # Ventana deslizante
    n = len(subgrupos)
    if window > n:
        window = n

    if titulo is None:
        titulo = (f"Carta S - {variable}   "
                  f"S̄={s_global:.3f}   UCLs={UCLs:.3f}   LCLs={LCLs:.3f}")

    def plot_window(start_idx: int):
        end_idx = min(start_idx + window, n)
        sg = subgrupos[start_idx:end_idx]
        vals = s_subgrupos.iloc[start_idx:end_idx]

        plt.figure(figsize=(12, 5))
        ax = plt.gca()

        # Zonas fuera de control
        ax.axhspan(UCLs, y_max, facecolor="red", alpha=0.08)
        ax.axhspan(y_min, LCLs, facecolor="red", alpha=0.08)

        # Serie S
        ax.plot(
            sg, vals,
            marker="o",
            linestyle="-",
            color="blue",
            label="S por subgrupo"
        )

        # Puntos fuera de control (fuera de UCLs / LCLs)
        mask_out = (vals > UCLs) | (vals < LCLs)
        ax.scatter(
            sg[mask_out],
            vals[mask_out],
            s=50,
            color="red",
            edgecolors="black",
            zorder=5,
            label="Fuera de control" if mask_out.any() else None
        )

        # Media global de S (verde punteada)
        ax.axhline(
            y=s_global,
            linestyle="--",
            linewidth=0.8,
            color="green",
            label="S̄ global"
        )

        # Límites de control (naranja punteado)
        ax.axhline(
            y=UCLs,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="UCLs"
        )
        ax.axhline(
            y=LCLs,
            linestyle="--",
            linewidth=0.8,
            color="orange",
            label="LCLs"
        )

        # Ejes y título
        ax.set_xlabel("Subgrupos")
        ax.set_ylabel(f"S de {variable}")
        ax.set_title(titulo)

        # Eje X acotado a la ventana actual
        ax.set_xlim(sg.min() - 0.5, sg.max() + 0.5)
        ax.set_xticks(sg)

        # Escala vertical
        ax.set_ylim(y_min, y_max)

        # Grid
        ax.grid(True, linestyle="--", alpha=0.4)

        # Marco exterior negro
        for spine in ax.spines.values():
            spine.set_edgecolor("black")
            spine.set_linewidth(1.2)

        # Leyenda fuera de la gráfica
        ax.legend(
            loc="upper center",
            bbox_to_anchor=(0.5, -0.20),
            ncol=4,
            frameon=False,
            fontsize=9
        )

        plt.tight_layout()
        plt.show()

    # Slider para elegir la ventana de subgrupos
    max_start = max(n - window, 0)
    slider = IntSlider(
        value=0,
        min=0,
        max=max_start,
        step=1,
        description="Inicio:",
        continuous_update=False,
    )

    interact(plot_window, start_idx=slider)


def selector_carta_s(df_s, s_prom, df_stats, window=30):
    variables = list(df_s.columns)

    def _plot(variable):
        graficar_carta_s(df_s, s_prom, df_stats, variable, window=window)

    interact(_plot, variable=Dropdown(options=variables, description="Variable:"))

EJECUCIÓN DEL VISUALIZADOR

In [ ]:
# EJECUCIÓN

# Lee todas las líneas del archivo
lines = leer_lineas(RUTA_IN, encoding=ENCODING_ALS)
print(f"NÚMERO TOTAL DE LÍNEAS EN EL ARCHIVO: {len(lines)}\n")

# Información contextual
df_info = parse_info_contextual(lines, idx_info=idx_info)
print("INFORMACIÓN CONTEXTUAL:")
display(df_info)

# Estadísticos
df_stats = parse_estadisticos(lines, idx_start=idx_stats_start, idx_end=idx_stats_end)
#print("\nTABLA ESTADÍSTICOS PRINCIPALES (Vista rápida):")
#display(df_stats.head())

# Datos crudos
df_raw = parse_datos_crudos(lines, idx_start=idx_raw_start)
#print("\nDATOS CRUDOS MEDIDOS POR EL ALS (Vista rápida):")
#display(df_raw.head())


In [ ]:
# EJECUCIÓN
df_resumen = estadisticos_resumen(df_stats) # Estadísticos principales de interés para seguimientos
df_resumen

In [ ]:
# EJECUCIÓN
df_limpio, df_xbar, df_s, xbar_prom, s_prom = procesar_datos_spc(df_raw) # Procesamiento de datos y cálculos necesarios


In [ ]:
# EJECUCIÓN
selector_carta_i(df_limpio, df_stats, window=20) # Graficar la carta I de la variable seleccionada en el desplegable


In [ ]:
# EJECUCIÓN
selector_carta_x(df_xbar, xbar_prom, df_stats, window=30) # Graficar la carta X de la variable seleccionada en el desplegable


In [ ]:
# EJECUCIÓN
selector_carta_s(df_s, s_prom, df_stats, window=30) # Graficar la carta S de la variable seleccionada en el desplegable


In [ ]:
# @title
# EJECUCIÓN: graficar la carta I de cada variable independiente
graficar_carta_i(df_limpio, df_stats, "t4012 [s]")
graficar_carta_i(df_limpio, df_stats, "t4018 [s]")
graficar_carta_i(df_limpio, df_stats, "t4015 [s]")
graficar_carta_i(df_limpio, df_stats, "V4062 [cmł]")
graficar_carta_i(df_limpio, df_stats, "p4072 [bar]")
graficar_carta_i(df_limpio, df_stats, "V4065 [cmł]")



In [ ]:
# @title
# EJECUCIÓN: graficar la carta X de cada variable independiente
graficar_carta_x(df_xbar, xbar_prom, df_stats, "t4012 [s]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "t4018 [s]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "t4015 [s]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "V4062 [cmł]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "p4072 [bar]")
graficar_carta_x(df_xbar, xbar_prom, df_stats, "V4065 [cmł]")

In [ ]:
# @title
# EJECUCIÓN: graficar la carta S de cada variable independiente
graficar_carta_s(df_s, s_prom, df_stats, "t4012 [s]")
graficar_carta_s(df_s, s_prom, df_stats, "t4012 [s]")
graficar_carta_s(df_s, s_prom, df_stats, "t4018 [s]")
graficar_carta_s(df_s, s_prom, df_stats, "t4015 [s]")
graficar_carta_s(df_s, s_prom, df_stats, "V4062 [cmł]")
graficar_carta_s(df_s, s_prom, df_stats, "p4072 [bar]")
graficar_carta_s(df_s, s_prom, df_stats, "V4065 [cmł]")


#### **OPTIMIZADOR DE TOLERANCIAS**

En este módulo se desarrolla un optimizador automático de límites de control (LSL y USL) para los planes de control del ALS, basado en el análisis estadístico de los datos históricos del proceso.
El módulo identifica condiciones de operación estables, filtra valores atípicos y propone nuevos límites de especificación utilizando criterios estadísticos, con el fin de reducir tolerancias excesivamente amplias y estandarizar el ajuste de los planes de control bajo una metodología trazable y basada en datos.

**Funciones auxiliares**

In [ ]:
# @title
# Configuración Default del optimizador de tolerancias

OPT_CFG_DEFAULT = {
    "usar_subgrupos_estables": True,   # usar info de df_xbar/df_s + límites de control
    "metodo_outliers": "iqr",          # Definir método "iqr" o "percentiles"
    "iqr_factor": 1.5,                 # Valor común 1.5·IQR
    "p_low": 0.5,                      # Percentil inferior para filtro (si se usa percentiles)
    "p_high": 99.5,                    # Percentil superior para filtro (si se usa percentiles)
    "cov_objetivo": 0.99,              # Cobertura mínima de limites recom, al menos 99% de los datos
    "k_sigma": None,                   # Si se quiere usar μ ± k·σ en vez de percentiles se define k=3 (Si None -> se usa percentiles)
    "min_frac_rango": 0.5,             # No reducir el rango a menos del 50% del rango actual
    "cpk_min": 1.33,                   # Si Cpk < 1.33, no estrechar tolerancias
    "min_n_datos": 50,                 # Mínimo de datos por variable para proponer algo
    "allow_gap": 1,
    "min_len_tramo": 25,
}
# FUNCIONES: Filtra tramo de subgrupos consecutivos estables
def _filtrar_subgrupos_estables(
    df_xbar,
    df_s,
    df_stats,
    var: str,
    allow_gap: int = 1,
    min_len: int = 100,
):
    """
    Devuelve el TRAMO CONSECUTIVO más largo de subgrupos estables por variable.

    Estable (por subgrupo) si:
      - X̄ está entre [LIIx, LISx]
      - S  está entre [LIIs, LISs]

    allow_gap:
      - 0  -> no permite datos fuera de especificación
      - 1  -> permite un dato aislado FALSE rodeado de datos TRUE
      - 2  -> permite hasta 2 datos FALSE aislador (más permisivo)
      * FALSE: Dato fuera de límites , TRUE: Dato dentro de límites
    min_len:
      - Longitud mínima deseada del tramo. Si no existe, devuelve igual el tramo más largo.
    """

    # Validaciones mínimas
    if (var not in df_xbar.columns) or (var not in df_s.columns) or (var not in df_stats.columns):
        return np.array([], dtype=df_xbar.index.dtype)

    # Extrae límites de control desde df_stats
    try:
        UCLx = float(df_stats.loc["LISx", var])
        LCLx = float(df_stats.loc["LIIx", var])
        UCLs = float(df_stats.loc["LISs", var])
        LCLs = float(df_stats.loc["LIIs", var])
    except KeyError:
        return np.array([], dtype=df_xbar.index.dtype)

    x = df_xbar[var].to_numpy()
    s = df_s[var].to_numpy()
    subgrupos = df_xbar.index.to_numpy()

    # Máscara base de estabilidad por subgrupo
    mask = (x >= LCLx) & (x <= UCLx) & (s >= LCLs) & (s <= UCLs)

    if not mask.any():
        return np.array([], dtype=subgrupos.dtype)

    # Repite allow_gap veces por si hay patrones tipo True False False True con allow_gap=2.
    if allow_gap > 0 and len(mask) >= 3:
        mask2 = mask.copy()
        for _ in range(allow_gap):
            for i in range(1, len(mask2) - 1):
                if (not mask2[i]) and mask2[i - 1] and mask2[i + 1]:
                    mask2[i] = True
        mask = mask2

    # Busca tramo consecutivo más largo de True
    best_start = 0
    best_len = 0
    cur_start = None
    cur_len = 0

    for i, ok in enumerate(mask):
        if ok:
            if cur_start is None:
                cur_start = i
                cur_len = 1
            else:
                cur_len += 1
        else:
            if cur_start is not None:
                if cur_len > best_len:
                    best_len = cur_len
                    best_start = cur_start
                cur_start = None
                cur_len = 0

    # Cierre si termina en True
    if cur_start is not None and cur_len > best_len:
        best_len = cur_len
        best_start = cur_start

    tramo = subgrupos[best_start: best_start + best_len]

    return tramo


# FUNCIONES: Filtración de datos más extremos
def _filtrar_outliers_serie(x: pd.Series, metodo="iqr", iqr_factor=1.5,
                            p_low=0.5, p_high=99.5):
    """
    Devuelve una serie filtrada eliminando datos más extremos (Outliers).
    - metodo="iqr": usa Q1-1.5·IQR y Q3+1.5·IQR
    - metodo="percentiles": usa [p_low, p_high]
    """
    x = x.dropna()
    if x.empty:
        return x
        # Rango intercuatil
    if metodo == "iqr":
        q1 = x.quantile(0.25)
        q3 = x.quantile(0.75)
        iqr = q3 - q1
        low = q1 - iqr_factor * iqr
        high = q3 + iqr_factor * iqr
    else:  # Percentiles
        low = x.quantile(p_low / 100.0)
        high = x.quantile(p_high / 100.0)

    return x[(x >= low) & (x <= high)] # Devuelve los valores medidos para comparación con tramo


**Función principal**

In [ ]:
# @title
# FUNCIONES: Cuerpo del optimizador de tolerancias

def optimizar_tolerancias(
    df_limpio: pd.DataFrame,
    df_stats: pd.DataFrame,
    df_xbar: pd.DataFrame = None,
    df_s: pd.DataFrame = None,
    variables: list[str] | None = None,
    config: dict | None = None,
) -> pd.DataFrame:

    # Se llama la configuración inicial
    cfg = OPT_CFG_DEFAULT.copy()
    if config is not None:
        cfg.update(config)

    metodo_outliers = cfg["metodo_outliers"]
    iqr_factor = cfg["iqr_factor"]
    p_low_cfg = cfg["p_low"]
    p_high_cfg = cfg["p_high"]
    cov_obj = cfg["cov_objetivo"]
    k_sigma = cfg["k_sigma"]
    min_frac_rango = cfg["min_frac_rango"]
    cpk_min = cfg["cpk_min"]
    min_n_datos = cfg["min_n_datos"]

    # Variables a analizar
    if variables is None:
        # Se excluyen las columnas que no son de interés, dejando solo las variables controladas
        cols_excluir = {"Muestra aleatoria", "Momento", "Cantidad piezas", "subgrupo", "pos"}
        variables = [
            c for c in df_limpio.columns
            if c not in cols_excluir and pd.api.types.is_numeric_dtype(df_limpio[c])
        ]

    filas_resultado = []

    for var in variables:
        comentario = []

        # Si la variable no esta en df_limpio
        if var not in df_limpio.columns:
            comentario.append("Variable no encontrada en df_limpio.")
            filas_resultado.append({
                "Variable": var,
                "Nominal": np.nan,
                "Media": np.nan,
                "Sigma": np.nan,
                "LSL Actual": np.nan,
                "USL Actual": np.nan,
                "LSL Recom": np.nan,
                "USL Recom": np.nan,
                "Rango Actual": np.nan,
                "Rango Recom": np.nan,
                "% Reduccion Rango": np.nan,
                "% Cobertura datos": np.nan,
                "Comentarios": "; ".join(comentario),
            })
            continue

        # Uso de la funcion para filtrar tramo de subgrupos consecutivos estables
        subgrupos_tramo = None
        if cfg["usar_subgrupos_estables"] and df_xbar is not None and df_s is not None:
            if "subgrupo" in df_limpio.columns:
                subgrupos_tramo = _filtrar_subgrupos_estables(
                    df_xbar=df_xbar,
                    df_s=df_s,
                    df_stats=df_stats,
                    var=var,
                    allow_gap=cfg.get("allow_gap", 1),
                    min_len=cfg.get("min_len_tramo", 25),
                )

        serie = df_limpio[var].dropna()

        # Se compara tramo estable con los valores medidos correspondientes
        if subgrupos_tramo is not None and len(subgrupos_tramo) > 0:
            serie = serie[df_limpio["subgrupo"].isin(subgrupos_tramo)]
        else:
            comentario.append("No se detectó tramo estable para esta variable.")
            # No optimiza si no hay tramo estable

        # No optimiza si no hay suficientes datos
        if len(serie) < min_n_datos:
            comentario.append(f"No hay datos suficientes (n={len(serie)} < {min_n_datos}).")

            # Se reportan solo los valores encontrados
            try:
                nominal = float(df_stats.loc["Valor nominal", var])
                LSL_act = float(df_stats.loc["Tolerancia inferior", var])
                USL_act = float(df_stats.loc["Tolerancia superior", var])
            except Exception:
                nominal = LSL_act = USL_act = np.nan

            filas_resultado.append({
                "Variable": var,
                "Nominal": nominal,
                "Media": np.nan,
                "Sigma": np.nan,
                "LSL Actual": LSL_act,
                "USL Actual": USL_act,
                "LSL Recom": np.nan,
                "USL Recom": np.nan,
                "Rango Actual": (USL_act - LSL_act) if np.isfinite(USL_act) and np.isfinite(LSL_act) else np.nan,
                "Rango Recom": np.nan,
                "% Reduccion Rango": np.nan,
                "% Cobertura datos": np.nan,
                "Comentarios": "; ".join(comentario),
            })
            continue

        # Filtrar outliers
        serie_filtrada = _filtrar_outliers_serie(
            serie,                   # Serie del tramo estable
            metodo=metodo_outliers,
            iqr_factor=iqr_factor,
            p_low=p_low_cfg,
            p_high=p_high_cfg,
        )

        # Cálculo del porcentaje de datos retenidos y eliminados después de filtrar outliers
        n_total = len(serie)
        n_filtrados = len(serie_filtrada)

        porc_retenido = 100 * n_filtrados / n_total
        porc_eliminado = 100 - porc_retenido

        # No optimiza si no quedan suficientes datos luego de filtrar outliers
        if len(serie_filtrada) < min_n_datos:
            comentario.append(
                f"Tras filtrar outliers, quedan pocos datos (n={len(serie_filtrada)})."
            )

        # Estadísticos del proceso del tramo estable
        mu = float(serie_filtrada.mean())
        sigma = float(serie_filtrada.std(ddof=1)) if len(serie_filtrada) > 1 else 0.0

        # Percentiles para info de referencia
        p_low_val = float(serie_filtrada.quantile(p_low_cfg / 100.0))
        p_high_val = float(serie_filtrada.quantile(p_high_cfg / 100.0))

        # Percentiles objetivo según cobertura deseada
        q_low_obj = (1.0 - cov_obj) / 2.0
        q_high_obj = 1.0 - q_low_obj
        p_low_obj_val = float(serie_filtrada.quantile(q_low_obj))
        p_high_obj_val = float(serie_filtrada.quantile(q_high_obj))

        # Lee valor nominal y tolerancias actuales
        try:
            nominal = float(df_stats.loc["Valor nominal", var])
        except Exception:
            nominal = mu  # si no hay nominal en df_stats, se usa mu

        try:
            LSL_act = float(df_stats.loc["Tolerancia inferior", var])
            USL_act = float(df_stats.loc["Tolerancia superior", var])
        except Exception:
            LSL_act = np.nan
            USL_act = np.nan

        rango_actual = (USL_act - LSL_act) if np.isfinite(USL_act) and np.isfinite(LSL_act) else np.nan

        # Lee Cpk para decidir si se puede estrechar (Cpk min = 1.33)
        Cpk_act = None
        for nombre_fila in ["Cpk", "Cpd", "Cpkx"]:
            if nombre_fila in df_stats.index:
                try:
                    Cpk_act = float(df_stats.loc[nombre_fila, var])
                    break
                except Exception:
                    continue

        if Cpk_act is not None and Cpk_act < cpk_min:
            comentario.append(f"Cpk actual bajo ({Cpk_act:.2f} < {cpk_min}), "
                              "no se recomienda estrechar la tolerancia.")
            # En este caso, simplemente reportamos y no cambiamos nada
            filas_resultado.append({
                "Variable": var,
                "Nominal": nominal,
                "Media": mu,
                "Sigma": sigma,
                "LSL Actual": LSL_act,
                "USL Actual": USL_act,
                "LSL Recom": LSL_act,
                "USL Recom": USL_act,
                "Rango Actual": rango_actual,
                "Rango Recom": rango_actual,
                "% Reduccion Rango": 0.0,
                "% Cobertura datos": np.nan,
                "% Datos Eliminados": porc_eliminado,
                "% Datos Retenidos": porc_retenido,
                "Comentarios": "; ".join(comentario),
            })
            continue

        # Cálculos de LSL/USL recomendados
        if k_sigma is not None and sigma > 0:
            # Basado en μ ± k·σ
            LSL_prop = mu - k_sigma * sigma
            USL_prop = mu + k_sigma * sigma
            comentario.append(f"Propuesta basada en μ ± {k_sigma}·σ.")
        else:
            # Basado en percentiles para cov_objetivo (99%)
            LSL_prop = p_low_obj_val
            USL_prop = p_high_obj_val
            comentario.append(f"Propuesta basada en percentiles que cubren ~{cov_obj*100:.1f}% de los datos.")

        rango_prop = USL_prop - LSL_prop

        # Cobertura real de los datos filtrados con la propuesta inicial
        dentro = ((serie_filtrada >= LSL_prop) & (serie_filtrada <= USL_prop)).mean()
        cobertura_pct = float(dentro * 100.0)

        # Candado: no reducir demasiado el rango respecto al actual (Se recomienda reducir progresivamente entre lotes)
        if np.isfinite(rango_actual) and rango_actual > 0:
            rango_min = min_frac_rango * rango_actual
            if rango_prop < rango_min:
                comentario.append(
                    f"Rango propuesto ({rango_prop:.4g}) < {min_frac_rango*100:.0f}% del rango actual "
                    f"({rango_actual:.4g}). Se limita el cierre."
                )
                # Recentrar rango mínimo sobre el nominal
                LSL_prop = nominal - rango_min / 2.0
                USL_prop = nominal + rango_min / 2.0
                rango_prop = rango_min

                # Recalcular cobertura con este rango ajustado
                dentro = ((serie_filtrada >= LSL_prop) & (serie_filtrada <= USL_prop)).mean()
                cobertura_pct = float(dentro * 100.0)

        # Construcción de tabla final de resultados de la optimización
        reduccion_pct = np.nan
        if np.isfinite(rango_actual) and rango_actual > 0:
            reduccion_pct = 100.0 * (1.0 - rango_prop / rango_actual)

        filas_resultado.append({
            "Variable": var,
            "Nominal": nominal,
            "Media": mu,
            "Sigma": sigma,
            "LSL Actual": LSL_act,
            "USL Actual": USL_act,
            "LSL Recom": LSL_prop,
            "USL Recom": USL_prop,
            "Rango Actual": rango_actual,
            "Rango Rec": rango_prop,
            "% Reduccion Rango": reduccion_pct,
            "% Cobertura límites": cobertura_pct,
            "Datos Filtrados": n_filtrados,
            "Datos Eliminados": n_total - n_filtrados,
            "% Datos Filtrados": porc_retenido,
            "% Datos Eliminados": porc_eliminado,

            "Comentarios": "; ".join(comentario),
        })

    df_res = pd.DataFrame(filas_resultado)
    return df_res


EJECUCIÓN DEL OPTIMIZADOR DE TOLERANCIAS

In [ ]:
# EJECUCIÓN: Optimizador de tolerancias

variables = ["t4012 [s]", "t4018 [s]", "t4015 [s]", "V4062 [cmł]", "p4072 [bar]", "V4065 [cmł]"]

df_tol_opt = optimizar_tolerancias(
    df_limpio=df_limpio,
    df_stats=df_stats,
    df_xbar=df_xbar,
    df_s=df_s,
    variables=variables,
    config={
        # Aqui podemos cambiar configuraciones sin alterar la Default
        "metodo_outliers": "percentiles",
        "p_low": 0.5,
        "p_high": 99.5,
        "cov_objetivo": 0.99,
        "min_frac_rango": 0.5,
        "cpk_min": 1.33,
    }
)

df_tol_opt


#### **SIMULADOR DE CAPACIDAD DEL PROCESO**

Este módulo permite simular el comportamiento de un plan de control bajo nuevos valores de límites (LSL y USL), utilizando datos históricos del proceso registrados en el ALS.
A partir de los valores nominales y estadísticos del proceso, se recalculan indicadores de desempeño como Cp, Cpk y desviación, permitiendo evaluar de forma anticipada el impacto de los nuevos límites antes de su aplicación en planta.

In [ ]:
# @title
# FUNCIONES: Cuerpo del simulador

def _ingresar_limites(msg: str) -> float:
    """Pide un número por input() y lo convierte a float. Acepta coma o punto decimal."""
    while True:
        s = input(msg).strip().replace(",", ".")
        try:
            return float(s)
        except ValueError:
            print("❌ ¡Valor inválido! Intenta de nuevo.")

def simulador_capacidad(df_stats: pd.DataFrame, variables=None) -> pd.DataFrame:
    """
    Simulador:
    - Extrae de df_stats: Valor nominal, xqq (media) y Sigma (sigma_w)
    - Pide por input() LSL y USL para cada variable.
    - Calcula: Rango, Cp, Cpk y Desviación
    - Devuelve tabla tipo resumen para compararación.
    """

    # Variables a simular
    if variables is None:
        variables = list(df_stats.columns)

    filas = []

    print("\n=== SIMULADOR DE CAPACIDAD DEL PROCESO ===")
    print("\nIngrese los valores de LSL y USL por variable.")
    print("Acepta números separados con coma o punto.\n")

    for var in variables:
        # Extrae datos base desde df_stats
        try:
            nominal = float(df_stats.loc["Valor nominal", var])
        except Exception:
            nominal = np.nan

        try:
            media = float(df_stats.loc["xqq", var])
        except Exception:
            media = np.nan

        try:
            sigma_w = float(df_stats.loc["Sigma", var])
        except Exception:
            sigma_w = np.nan

        # Pide límites por consola mediante input()
        print(f"Variable: {var}")
        LSL = _ingresar_limites("  Ingrese el límite inferior (LSL): ")
        USL = _ingresar_limites("  Ingrese el límite superior (USL): ")

        # Validación de consistencia de límites
        if USL <= LSL:
            print("⚠️¡ERROR! Límite inferior debe ser menor al límite superior (USL <= LSL)\n")

        # Cálculos
        rango = USL - LSL if USL > LSL else np.nan

        if (USL > LSL) and np.isfinite(sigma_w) and sigma_w > 0 and np.isfinite(media):
            Cp = (USL - LSL) / (6.0 * sigma_w)
            Cpu = (USL - media) / (3.0 * sigma_w)
            Cpl = (media - LSL) / (3.0 * sigma_w)
            Cpk = min(Cpu, Cpl)

            Desv = (Cp - Cpk) / Cp * 100 if Cp != 0 else np.nan
        else:
            Cp = np.nan
            Cpk = np.nan
            Desv = np.nan

        filas.append({
            "Variables": var,
            "Rango": rango,
            "Valor nominal": nominal,
            "Media": media,
            "Sigma_w": sigma_w,
            "Cp": Cp,
            "Cpk": Cpk,
            "Desviación": Desv
        })

        print("")  # Espacio entre variables para mejor visualización

    df_sim = pd.DataFrame(filas)

    # Redondeo de valores
    cols_round = ["Rango", "Valor nominal", "Media", "Sigma_w", "Cp", "Cpk", "Desviación"]
    for c in cols_round:
        df_sim[c] = pd.to_numeric(df_sim[c], errors="coerce").round(6)

    return df_sim


EJECUCIÓN DEL SIMULADOR DE CAPACIDAD

In [ ]:
# EJECUCIÓN: simula todas las variables de control
df_sim = simulador_capacidad(df_stats)
df_sim